In [30]:
# Step 1: Imports
import pennylane as qml
import numpy as np
import jax
from jax import numpy as jnp
from pennylane import qchem
import time

jax.config.update("jax_enable_x64", True)

# Step 2: Define symbols and geometry
symbols = ["Li", "H"]
geometry = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 2.969280527]])


In [31]:

# Step 3: Build Hamiltonian (fixed to pass symbols, geometry separately)
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=2,
    active_orbitals=5
)

# Step 4: Get excitations
active_electrons = 2
singles, doubles = qchem.excitations(active_electrons, qubits)

print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 24


Adaptive Optimizer
- grows an input quantum circuit by adding and optimizing gates selected from a user-defined collection of operators
- algo first appends all gates provided in the initial operator pool and computes the circuit gradients with respect to the gate parameters.
- it retains the gate which has the largest gradient and then optimizes its parameter
- the process of growing the circuit can be repeated until the computed gradients converge to zero
- use Adaptive Optimizer to perform an ADAPT-VQE simulation and build an adaptive circuit for LiH

In [ ]:
# Step 5: Build operator pool
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations


In [33]:
# Step 6: Define Hartree-Fock initial state circuit
hf_state = qchem.hf_state(active_electrons, qubits)

dev = qml.device("default.qubit", wires=qubits)

@qml.qnode(dev)
def circuit():
    for i in np.nonzero(hf_state)[0]:
        qml.PauliX(i)
    return qml.expval(H)

In [34]:

# Step 7: Adaptive optimizer
opt = qml.optimize.AdaptiveOptimizer()

for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
    
    if i % 3 == 0:
        print(f"n = {i},  E = {energy:.8f} H, Largest Gradient = {gradient:.3f}")
        print(qml.draw(circuit, decimals=None)())
        print()
        
    if gradient < 3e-3:
        break

n = 0,  E = -7.86266588 H, Largest Gradient = 0.124
0: ──X─╭G²─┤ ╭<𝓗>
1: ──X─├G²─┤ ├<𝓗>
2: ────│───┤ ├<𝓗>
3: ────│───┤ ├<𝓗>
4: ────│───┤ ├<𝓗>
5: ────│───┤ ├<𝓗>
6: ────│───┤ ├<𝓗>
7: ────│───┤ ├<𝓗>
8: ────├G²─┤ ├<𝓗>
9: ────╰G²─┤ ╰<𝓗>

n = 3,  E = -7.88038372 H, Largest Gradient = 0.020
0: ──X─╭G²─╭G²─╭G²─╭G²─┤ ╭<𝓗>
1: ──X─├G²─├G²─├G²─├G²─┤ ├<𝓗>
2: ────│───│───├G²─│───┤ ├<𝓗>
3: ────│───├G²─│───│───┤ ├<𝓗>
4: ────│───│───│───│───┤ ├<𝓗>
5: ────│───│───│───│───┤ ├<𝓗>
6: ────│───│───│───├G²─┤ ├<𝓗>
7: ────│───│───│───╰G²─┤ ├<𝓗>
8: ────├G²─╰G²─│───────┤ ├<𝓗>
9: ────╰G²─────╰G²─────┤ ╰<𝓗>

n = 6,  E = -7.88189085 H, Largest Gradient = 0.006
0: ──X─╭G²─╭G²─╭G²─╭G²─╭G²─╭G²─╭G²─┤ ╭<𝓗>
1: ──X─├G²─├G²─├G²─├G²─├G²─├G²─├G²─┤ ├<𝓗>
2: ────│───│───├G²─│───│───├G²─├G²─┤ ├<𝓗>
3: ────│───├G²─│───│───│───╰G²─│───┤ ├<𝓗>
4: ────│───│───│───│───├G²─────│───┤ ├<𝓗>
5: ────│───│───│───│───╰G²─────│───┤ ├<𝓗>
6: ────│───│───│───├G²─────────│───┤ ├<𝓗>
7: ────│───│───│───╰G²─────────│───┤ ├<𝓗>
8: ────├G²─╰G²─│──────────